# Imports

In [1]:
from typing import List, Dict
from bs4 import BeautifulSoup
from urllib.parse import urljoin
from pathlib import Path
import pandas as pd
from validate_docbr import CNPJ
import zipfile

# Transformação e validação de dados

### Validacao dos Dados
- CNPJ inválido = SEPARADO PARA OUTRO DF -> NULO = invalido
- Valores negativos = EXCLUIDO (Ja realizado na etapa anterior) 
- Razão social vazia = EXCLUIR 

In [2]:
df_consolidado = pd.read_csv("downloads/2025/extraido/consolidado/consolidado_despesas.csv")
df_consolidado.info()

<class 'pandas.DataFrame'>
RangeIndex: 2113924 entries, 0 to 2113923
Data columns (total 5 columns):
 #   Column         Dtype  
---  ------         -----  
 0   ValorDespesas  float64
 1   Ano            int64  
 2   Trimestre      int64  
 3   CNPJ           float64
 4   Razao_Social   str    
dtypes: float64(2), int64(2), str(1)
memory usage: 80.6 MB


In [ ]:
cnpj_validator = CNPJ()

# Validar CNPJ
def is_valid_cnpj(cnpj_value):
    try:
        # Convert to string, remove decimals, and pad to 14 digits
        cnpj_str = str(int(cnpj_value)).zfill(14)
        return cnpj_validator.validate(cnpj_str)
    except:
        return False

# CNPJ invalidos
df_invalid_cnpj = df_consolidado[~df_consolidado['CNPJ'].apply(is_valid_cnpj)].copy()

# CNPJ validdos
df_consolidado = df_consolidado[df_consolidado['CNPJ'].apply(is_valid_cnpj)].copy()

# ValorDespesas positivo 
# Razao_Social não vazia
df_consolidado = df_consolidado[(df_consolidado['ValorDespesas'] > 0) & 
                                (df_consolidado['Razao_Social'].notna()) & 
                                (df_consolidado['Razao_Social'].str.strip() != '')].copy()

df_consolidado['CNPJ'] = (
    df_consolidado['CNPJ']
    .astype(str)
    .str.replace(r'\.0$', '', regex=True)  # remove .0 se veio de float
    .str.zfill(14)
)

df_consolidado.info()
df_invalid_cnpj.info()

# Export to CSV
df_consolidado.to_csv('despesas_validas.csv', index=False)
df_invalid_cnpj.to_csv('despesas_invalidas_cnpj.csv', index=False)

<class 'pandas.DataFrame'>
Index: 964452 entries, 0 to 2113860
Data columns (total 5 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   ValorDespesas  964452 non-null  float64
 1   Ano            964452 non-null  int64  
 2   Trimestre      964452 non-null  int64  
 3   CNPJ           964452 non-null  str    
 4   Razao_Social   964452 non-null  str    
dtypes: float64(1), int64(2), str(2)
memory usage: 44.1 MB
<class 'pandas.DataFrame'>
Index: 18739 entries, 31496 to 1987353
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   ValorDespesas  18739 non-null  float64
 1   Ano            18739 non-null  int64  
 2   Trimestre      18739 non-null  int64  
 3   CNPJ           0 non-null      float64
 4   Razao_Social   0 non-null      str    
dtypes: float64(2), int64(2), str(1)
memory usage: 878.4 KB


In [4]:
df_invalid_cnpj.head()

,ValorDespesas,Ano,Trimestre,CNPJ,Razao_Social
31496,1362569.82,2025,1,NaN,NaN
31497,1362569.82,2025,1,NaN,NaN
31498,32000.00,2025,1,NaN,NaN
31499,0.00,2025,1,NaN,NaN
31500,0.00,2025,1,NaN,NaN


In [5]:
df_consolidado.head()

,ValorDespesas,Ano,Trimestre,CNPJ,Razao_Social
0,1070.00,2025,1,41511429000120,UNIMED REGIONAL DE FLORIANO - COOPERATIVA DE T...
1,1070.00,2025,1,41511429000120,UNIMED REGIONAL DE FLORIANO - COOPERATIVA DE T...
2,1070.00,2025,1,41511429000120,UNIMED REGIONAL DE FLORIANO - COOPERATIVA DE T...
3,1070.00,2025,1,41511429000120,UNIMED REGIONAL DE FLORIANO - COOPERATIVA DE T...
4,99024.72,2025,1,41511429000120,UNIMED REGIONAL DE FLORIANO - COOPERATIVA DE T...


### Enriquecimento de Dados
- Ja foi baixado para conseguir o CNPJ
- JOIN de novo usando CNPJ como chave
- Adicionar colunas: RegistroANS, Modalidade e UF 
- Análise crítica: Você encontrará CNPJs no arquivo consolidado que não existem no
cadastro (ou vice-versa). 
    ->  Isso so vai ocorrer caso RIGHT no relatorio, trazendo os CNPJ que ainda não foram usados/vistos
    - Registros sem match no cadastro - ??
    - CNPJs que aparecem múltiplas vezes no cadastro com dados diferentes -> Normal, pois Tem cnpj com razao duplicado apenas no Merge com o relatório, pois existem contas contabeis diferentes, que não foram trazidas para o df final


- Trade-off técnico: Para o join, você precisará decidir como processar os dados.
Considere diferentes estratégias de processamento e escolha a que fizer mais sentido
para o seu contexto. Documente sua escolha e justifique baseado no tamanho
estimado dos dados e nas características do problema.

- Right para ter todos os registros possiveis baseados no CNPJ, por mais q não tenha as outras informaçoes do consolidado
    

In [6]:
df_relatorio = pd.read_csv(f"Relatorio_cadop.csv", sep=';', dtype=str, encoding='latin1')
df_relatorio.head()

,REGISTRO_OPERADORA,CNPJ,Razao_Social,Nome_Fantasia,Modalidade,Logradouro,Numero,Complemento,Bairro,Cidade,UF,CEP,DDD,Telefone,Fax,Endereco_eletronico,Representante,Cargo_Representante,Regiao_de_Comercializacao,Data_Registro_ANS
0,419761,19541931000125,18 DE JULHO ADMINISTRADORA DE BENEFÃCIOS LTDA,NaN,Administradora de BenefÃ­cios,RUA CAPITÃO MEDEIROS DE REZENDE,274,NaN,PRAÃA DA BANDEIRA,AlÃ©m ParaÃ­ba,MG,36660000,32,34624649,NaN,contabilidade@cbnassessoria.com.br,LUIZ HENRIQUE MARENDINO GONÃALVES,SÃCIO ADMINISTRADOR,6,2015-05-19
1,421545,22869997000153,2B ODONTOLOGIA OPERADORA DE PLANOS ODONTOLÃGI...,NaN,Odontologia de Grupo,RUA CATÃO,128,SALA 126,VILA ROMANA,SÃ£o Paulo,SP,05049000,11,34415852,NaN,labmarisol@gmail.com,MARISOL BECHELLI,SÃCIO ADMINISTRADORA,4,2019-06-13
2,421421,27452545000195,2CARE OPERADORA DE SAÃDE LTDA.,NaN,Medicina de Grupo,RUA: BERNARDINO DE CAMPOS,230,1Âº ANDAR,CENTRO,Campinas,SP,13010151,19,37901224,NaN,ans.plano@hospitalcare.com.br,RODRIGO PINHO RIBEIRO,REPRESENTANTE,5,2018-10-09
3,418030,13138885000131,A.P.S. ADMINISTRADORA DE BENEFÃCOS LTDA.,A.P.S. SAÃDE.,Administradora de BenefÃ­cios,RUA VOLUNTÃRIOS DA PÃTRIA,2525,CONJUNTO 143 - SALA 01,SANTANA,SÃ£o Paulo,SP,02401000,11,45223468,NaN,diretoria@apssaude.com.br,PERCÃVEL GAETA,SÃ³CIO-ADMINISTRADOR E REPRESEN,4,2011-05-05
4,314668,17505793000101,ABERTTA SAÃDE - ASSOCIAÃÃO BENEFICENTE DOS ...,ABERTTA SAÃDE,AutogestÃ£o,AV. BERNARDO MONTEIRO,831,"Subsolo, 2Âº andar e 3Âº andar",SANTA EFIGÃNIA,Belo Horizonte,MG,30150281,31,32484300,32484377,abertta.ans@arcelormittal.com.br,WERNER DUARTE DALLA,Diretor Presidente,4,1998-12-28


In [7]:
df_consolidado['CNPJ'] = df_consolidado['CNPJ'].astype('int64').astype(str)
df_relatorio['CNPJ'] = df_relatorio['CNPJ'].astype(str)

df_enriquecido = df_consolidado.merge(
    df_relatorio[['CNPJ', 'REGISTRO_OPERADORA', 'Modalidade', 'UF']]
        .rename(columns={'REGISTRO_OPERADORA': 'RegistroANS'}),
    how='right',
    on='CNPJ'
)

df_enriquecido.info()


<class 'pandas.DataFrame'>
RangeIndex: 709568 entries, 0 to 709567
Data columns (total 8 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   ValorDespesas  709039 non-null  float64
 1   Ano            709039 non-null  float64
 2   Trimestre      709039 non-null  float64
 3   CNPJ           709568 non-null  str    
 4   Razao_Social   709039 non-null  str    
 5   RegistroANS    709568 non-null  str    
 6   Modalidade     709568 non-null  str    
 7   UF             709568 non-null  str    
dtypes: float64(3), str(5)
memory usage: 43.3 MB


In [8]:
df_enriquecido.isna().sum()

ValorDespesas    529
Ano              529
Trimestre        529
CNPJ               0
Razao_Social     529
RegistroANS        0
Modalidade         0
UF                 0
dtype: int64

### Agregação com Múltiplas Estratégias
- Agrupe os dados por RazaoSocial e UF
- Calcule o total de despesas por operadora/UF, medias de despesas por trimestre para cada operadora/UF, Desvio padrão das despesas (para identificar operadoras com valores muito
variáveis)

- Trade-off técnico: Para ordenação, você precisará escolher uma estratégia
considerando o volume de dados e os recursos disponíveis. Justifique sua escolha no
README.

- Ordene por valor total (maior para menor)
- Salve o resultado em um novo CSV nomeado despesas_agregadas.csv
- Compacte o arquivo em Teste_{seu_nome}.zip

In [ ]:
# Agrupar por Razao_Social e UF, calcular total de despesas e desvio padrão
df_agg = df_enriquecido.groupby(['Razao_Social', 'UF']).agg(
    total_despesas=('ValorDespesas', 'sum'),
    std_despesas=('ValorDespesas', 'std')
).reset_index()

# Calcular média de despesas por trimestre para cada operadora/UF, média do primeiro, segundo e terceiro trimestres 
df_mean_trim = df_enriquecido.groupby(['Razao_Social', 'UF', 'Trimestre'])['ValorDespesas'].mean().reset_index()

columns = ['Razao_Social', 'UF']

for trimestre in df_mean_trim['Trimestre'].unique():
    columns.append(f'media_trimestre_{trimestre:.0f}')

# Pivotar para ter trimestres como colunas
df_pivot = df_mean_trim.pivot(index=['Razao_Social', 'UF'], columns='Trimestre', values='ValorDespesas').reset_index()
df_pivot.columns = columns


# Mesclar com df_agg
df_final = df_agg.merge(df_pivot, on=['Razao_Social', 'UF'], how='left')

# Ordenar por total_despesas decrescente
df_final = df_final.sort_values('total_despesas', ascending=False)

# Salvar em CSV
df_final.to_csv('despesas_agregadas.csv', index=False)

# Compactar em ZIP
with zipfile.ZipFile('Teste_Gabriel.zip', 'w') as zipf:
    zipf.write('despesas_agregadas.csv')

In [33]:
df_final.head()

,Razao_Social,UF,total_despesas,std_despesas,media_trimestre_1,media_trimestre_2,media_trimestre_3
96,BRADESCO SAÚDE S.A.,RJ,5.648629e+11,1.333377e+09,2.972927e+08,2.799009e+08,3.067912e+08
18,AMIL ASSISTÊNCIA MÉDICA INTERNACIONAL S.A.,SP,4.219167e+11,8.333890e+08,1.752787e+08,2.131357e+08,1.857467e+08
178,HAPVIDA ASSISTENCIA MEDICA S.A.,CE,2.504994e+11,3.995303e+08,9.151441e+07,1.129140e+08,9.861520e+07
226,NOTRE DAME INTERMÉDICA SAÚDE S.A.,SP,2.437911e+11,4.187863e+08,1.167453e+08,1.277346e+08,9.562129e+07
353,UNIMED BELO HORIZONTE COOPERATIVA DE TRABALHO ...,MG,1.210336e+11,1.621446e+08,2.837966e+07,3.003795e+07,3.109792e+07
